## OpenAI API 호출

In [1]:
from openai import OpenAI

OPENAI_API_KEY = "API KEY를 입력하세요"

client = OpenAI(api_key=OPENAI_API_KEY)

## 채팅 기반의 완성 생성, 모델=gpt-3.5

In [2]:
# 채팅 기반의 완성 생성, 모델='gpt-3.5'
# temperature는 생성할 응답의 창의성을 조절, 0은 최소한의 창의성
completion = client.chat.completions.create(model='gpt-3.5-turbo-0125',
                                            messages=[{'role': 'user', 'content': '왜 하늘은 하늘색인가요?'}],
                                            temperature=0.0)

answer = completion.choices[0].message.content
for sentence in answer.split('. '):
    print(sentence)

하늘은 하늘색인 이유는 대기 중의 분자들이 햇빛을 흡수하고 산란시키는 과정 때문입니다
태양으로부터 오는 햇빛은 다양한 파장을 가지고 있는데, 대기 중의 분자들은 이 햇빛을 흡수하고 다시 방출하면서 파장이 긴 파장인 파란색과 보라색을 제외한 파장들이 산란되어 우리 눈에는 파란색으로 보이게 됩니다
따라서 하늘은 파란색으로 보이게 되는 것입니다.


## Prompt는 2가지 종류

* System Prompt
    1. User Prompt를 전달하기 전에 맥락, 지침을 설정하는 프롬프트
    2. 페르소나, 어조 설정
    3. 출력값 지정, 페르소나 및 어조 설정, 외부 정보 주입, 규칙 설정

* User Prompt
    1. 사용자가 ChatGPT한테 실제로 전달

### System Prompt 호출

In [3]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[
        {'role': 'system', 'content': '당신은 물리선생님입니다. 저학년 학생들을 대상으로 친절하고 쉽게 설명해주세요.'}, # 역할(물리선생님), 맥락(저학년에게 친절하게)
        {'role': 'user', 'content': '왜 숲은 초록색인가요?'}],
        temperature=0.0
)

sys_answer = completion.choices[0].message.content
for sentence in sys_answer.split('. '):
    print(sentence)

숲이 초록색인 이유는 나무와 식물들이 속에 포함된 엽록소라는 물질 때문입니다
엽록소는 식물들이 태양빛을 흡수하여 에너지를 생산하는데 사용하는 역할을 합니다
이 과정을 광합성이라고 해요
엽록소는 태양빛을 흡수할 때 녹색 빛을 흡수하고 다른 색깔의 빛을 반사하기 때문에 우리 눈에는 초록색으로 보이게 됩니다
그래서 숲은 엽록소가 많이 포함되어 있어서 초록색으로 보이는 거죠.


### User Prompt 호출
* System X, User가 System과 같이 사용

In [4]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[
        {'role': 'user', 'content': '당신은 물리선생님입니다. 저학년 학생들을 대상으로 친절하고 쉽게 설명해주세요. 왜 숲은 초록색인가요?'}
    ],
    temperature=0.0
)

user_answer = completion.choices[0].message.content
for sentence in user_answer.split('. '):
    print(sentence)

숲이 초록색인 이유는 주로 나무와 식물들이 엽록소라는 녹색 색소를 가지고 있기 때문입니다
엽록소는 빛을 흡수하여 에너지를 생산하는 역할을 합니다
빛 중에서 파란색과 녹색의 파장을 가진 빛을 특히 잘 흡수하고, 빨간색의 파장을 가진 빛은 반사하기 때문에 우리 눈에 초록색으로 보입니다
따라서 숲은 주로 초록색으로 보이는 것이죠
또한 나무와 식물들이 많이 모여 있어서 눈에 초록색이 더 많이 보이기도 합니다
숲은 자연의 아름다움을 담고 있는 곳이기 때문에 초록색으로 더욱 아름다운 것 같아요.


* System Prompt, User Prompt 둘다 비슷한 답을 얻었다.
* User Prompt는 유저가 직접 적어야 하지만, System Prompt는 System 개발자가 직접 맞춰서 작성한 Prompt이기 때문에 개발자에 맞춰져있다.
* 그래도 자세히 보면 System Prompt는 User Prompt보다 저학년 학생들이 쉽게 이해할 수 있다고 생각한다.

### Completion X, Stream O
* Completion의 뜻은 '완성'으로, 결과를 한번에 받음
* Stream은 실시간으로 바로바로 받을 수 있다는 특징이 있음
* ChatGPT가 문장을 완성하지 않고, 각 단어별로 바로바로 보여주는 방법

In [5]:
stream = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': '왜 숲은 초록색인가?'}],
    temperature=0.0,
    stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end='')

숲이 초록색인 이유는 주로 나무와 식물들이 엽록소라는 녹색 색소를 가지고 있기 때문입니다. 엽록소는 빛을 흡수하여 에너지를 생산하는 역할을 하며, 녹색색소이기 때문에 숲이나 나무들이 초록색으로 보이게 됩니다. 또한, 빛이 나무나 식물들의 잎사귀에 반사되어 우리 눈에 초록색으로 보이기 때문에 숲은 주로 초록색으로 인식됩니다.

### 특징
1. Prompt는 영어로 해야 모델의 제성능을 발휘하는 편
2. AI 모델의 출력값은 입력값에 의존도가 매우 높음
3. Prompt를 이렇게 저렇게 바꿨을 때 "더 좋아보이는" 결과보다는 특정 지표에서 유의미하게 더 좋거나 여러 번의 블라인드 테스팅을 통해 더 좋은 Prompt를 정하는 것을 추천

## 프롬프트 평가
* LLM이 어느 정도 성능을 가진 모델인지 정량 or 정성적으로 확인하는 객관적인 지표
1. 사람
2. 모델
3. 코드

### 모델을 활용한 평가
1. G-Eval
2. MT-Bench

### GPT-3.5과 GPT-4의 질문에 대한 답변

In [6]:
# 질문
question = "왜 숲은 초록색인가?"

# gpt-3.5 모델을 통해 질문 후, 답변(answer3_5)을 얻음
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': question}],
    temperature=0.0
)

answer3_5 = completion.choices[0].message.content
for sentence in answer3_5.split('. '):
    print(sentence)

print('-'*30)

# gpt-4 모델을 통해 질문 후, 답변(answer4)을 얻음
completion = client.chat.completions.create(
    model='gpt-4-1106-preview', # 최근 모델
    messages=[{'role': 'user', 'content': question}],
    temperature=0.0
)

answer4 = completion.choices[0].message.content
print(answer4)

숲이 초록색인 이유는 주로 나무와 식물들이 엽록소라는 녹색 색소를 가지고 있기 때문입니다
엽록소는 빛을 흡수하여 에너지를 생산하는 역할을 하며, 녹색색소이기 때문에 숲이 초록색으로 보이게 됩니다
또한, 숲의 식물들이 많은 양의 엽록소를 함유하고 있기 때문에 숲 전체가 초록색으로 빛나는 것입니다
이러한 이유로 숲은 주로 초록색으로 보이게 됩니다.
------------------------------
숲이 초록색으로 보이는 주된 이유는 나무와 식물의 잎에 포함된 엽록소(chlorophyll) 때문입니다. 엽록소는 식물이 광합성을 통해 태양 에너지를 화학 에너지로 변환하는 데 필수적인 역할을 합니다. 광합성 과정에서 식물은 이산화탄소와 물을 사용하여 포도당과 같은 영양분을 만들고, 산소를 방출합니다.

엽록소는 빛의 스펙트럼 중에서 빨간색과 파란색 파장을 흡수하고, 초록색 파장을 반사합니다. 이 반사된 초록색 빛이 우리 눈에 들어와 식물의 잎이 초록색으로 보이게 만듭니다. 숲이 대부분 초록색으로 보이는 것은 나무와 식물의 잎이 많기 때문에, 이러한 초록색 반사가 매우 흔하기 때문입니다.

물론, 계절에 따라 나무의 잎 색깔이 변할 수 있습니다. 예를 들어, 가을에는 일부 나무의 잎이 노란색, 주황색, 빨간색으로 변하며, 이는 엽록소가 분해되고 다른 색소들이 더 잘 보이게 되기 때문입니다. 그러나 봄과 여름에는 엽록소의 활동이 가장 활발하여 숲이 풍부한 초록색을 띠게 됩니다.


### 평가 프롬프트

In [7]:
# 평가 프롬프트에 넣어보자
# 평가 프롬프트 출처: MT-Bench 논문 https://arxiv.org/pdf/2306.05685.pdf (A. Prompt Templates Figure 5)

prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Question]
{question}

[The Start of Assistant A’s Answer]
{answer3_5}
[The End of Assistant A’s Answer]

[The Start of Assistant B’s Answer]
{answer4}
[The End of Assistant B’s Answer]"""

print(prompt)

[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Question]
왜 숲은 초록색인가?

[The Start of Assistan

### 평가

In [8]:
# 평가
# 최신 모델 'gpt-4-turbo-2024-04-09'에 위 prompt를 넣어보자.
completion = client.chat.completions.create(
    model='gpt-4-turbo-2024-04-09', # 최신 모델
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Upon reviewing the responses from Assistant A and Assistant B, it is evident that both assistants provided accurate and relevant explanations for why forests appear green, primarily attributing this to the presence of chlorophyll in the leaves of trees and plants. However, there are notable differences in the depth and detail of their explanations.

Assistant A's response is straightforward and correct, explaining that chlorophyll absorbs light to produce energy, which results in the green color of forests. This answer is concise and directly addresses the user's question.

Assistant B, on the other hand, goes further in depth by explaining the specific process of photosynthesis and how chlorophyll plays a crucial role in converting sunlight into chemical energy. Assistant B also elaborates on how chlorophyll absorbs certain wavelengths of light while reflecting others, specifically the green wavelengths, which is why we perceive the leaves as green. Additionally, Assistant B adds info

```
영어 해석본을 번역해서 봐보자.

보조자 A와 보조자 B의 응답을 비교하면서 두 보조자 모두 식물 잎에서 엽록소의 역할에 초점을 맞춰 숲이 녹색으로 나타나는 이유에 대해 정확하고 관련성 있는 설명을 제공합니다. 그러나 설명의 깊이와 세부사항에는 차이가 있습니다.

보조자 A의 반응은 간결하며 빛의 흡수와 녹색 빛의 반사를 언급하면서 녹색의 원인이 엽록소임을 정확하게 식별합니다. 또한 녹색의 우세에 기여하는 숲의 식물 성장의 경쟁적 측면을 다루고 있습니다.

반면 보조자 B의 반응은 광합성 과정과 엽록소에 의해 흡수되고 반사되는 빛의 특정 파장에 대해 더 자세히 설명합니다. 이 보조원은 또한 잎의 다른 색소에 대한 엽록소의 지배력이 어떻게 녹색을 띠게 되는지에 대해 확장하고 잎의 계절적 색상 변화에 대해 더 자세히 설명하여 답변에 깊이와 교육적 가치를 더합니다.

창의성 측면에서 두 응답 모두 과학적인 설명을 고수하고 질문의 사실적 성격에 적합한 창의적 해석을 시도하지 않습니다.

전반적으로 보조 도구 B는 기본 질문을 해결할 뿐만 아니라 관련 현상(예: 잎 색깔의 계절적 변화)에 대한 사용자의 이해를 풍부하게 하는 보다 자세하고 포괄적인 설명을 제공합니다. 이렇게 하면 보조자 B의 응답이 더 유용하고 유익해집니다.

최종 평결: [[B]]

결론적으로 B예시가 더 좋은 예시라고 평가했다.
```

1. 평가 프롬프트에 'gpt-3.5'와 'gpt-4'의 질문한 답변을 넣는다.
2. 넣은 '평가 프롬프트'를 최신 gpt모델 'gpt-4-turbo-2024-04-09'에 평가한다.
3. 결론적으로 B예시(GPT-4)가 더 좋은 예시라고 하였다.
4. 최신 모델은 좋은거구나..

## 사람, 모델, 코드를 활용한 평가의 각각 장단점
### 1. 사람을 활용한 평가
- 장점
1. 복잡한 맥락과 뉘앙스를 사람은 이해할 수 있기 때문에, 심층적 이해와 맥락을 평가하는데 유리함
2. 통제된 환경일 경우, 사람이 직접 평가한 방법이라 신뢰를 할 수 있음
3. 실제 사용해보고 경험과 만족도를 반영할 수 있어, 사용자 중심의 평가 가능

- 단점
1. 다수의 의한 노이즈 발생 가능성이 있음
2. 전문가가 아닐시 정확성이 떨어짐
3. 시간과 비용이 높을 수 있음
### 2. 모델을 활용한 평가
-장점
1. 정량적 지표를 통해 객관적으로 성능을 평가할 수 있음
2. 자동으로 대규모 데이터셋을 평가할 수 있어, 시간과 비용을 절약할 수 있음
3. 평가의 일관성과 신뢰성을 높일 수 있음

- 단점
1. 모델은 복잡한 맥락이나 뉘앙스를 완벽히 이해하지 못할 수 있음
2. 편향이 평가 결과에 미칠 수 있음
### 3. 코드를 활용한 평가
- 장점
1. 무료로 평가할 수 있음
2. 신속하고 정확하게 평가할 수 있음
3. 복잡한 계산 가능

- 단점
1. 신뢰성이 떨어질 수 있음
2. 기술적 제약 발생

### 요약
1. 태스크에 적합한 사람이 평가하는 방법이 제일 좋음
2. 현실적으로 사람이 평가하는 방법은 쉽지 않아, 모델이 평가하는 방법도 충분히 사용함
3. 코드를 활용한 평가를 권장, 현실적으로 고려하는 점이 '비용'이기 때문에
4. 가장 이상적인 평가 방법은 정량적 + 정성적 평가 모두 하는 법
5. 최종 지표는 사용자의 피드백

### 정리
- 평가 기준은 총 3가지
1. 사람을 활용한 평가
2. 모델을 활용한 평가
3. 코드를 활용한 평가

## 프롬프트 엔지니어링 기법들

### Zero-Shot Prompting
* 특정 작업에 대해 명시적으로 훈련받지 않은 상태에서, 모델이 주어진 프롬프트를 통해 바로 작업을 수행하도록 유도하는 것

### 대표적인 기법 Few Shot, Chain of Thought
* 현재 Chain of Thought 기법을 많이 씀
* 어느 기법이 좋다는 것을 없고, 각 기법을 잘 이해하고 원리를 아는게 중요

### 1. Few Shot
* 답변을 얻기 위해 참고할 수 있는 정답 사례를 알려줘 프롬프트 기능을 올려주는 방법
* Q1 : ~~~~? (참고)
* A1 : ~~~ (참고)
* Q2 : ~~~?
* A2 :
* 관련 논문 : https://arxiv.org/abs/2005.14165 (=GPT-3 논문)
* 3-shot의 경우 정답 사례를 프롬프트에 3개 입력
* 공인된 프롬프트 엔지니어링 방법론, 평가에서도 사용, 모델 사이즈의 크기가 어느정도 있어야 함

### 1-1. Few Shot이 잘 동작하는 이유
* 추가학습없이 정답 사례만 추가해도 성능이 급격히 증가 (파라미터를 수정하지 않아도)
* 사전훈련단계에서 잘 훈련된 언어모델
* 태스크에 빠르게 적응
* In-Context Learning이라고 부름

### Few Shot (기본 프롬프트)

In [9]:
prompt = """Q: who wrote the book The Little Prince?"""

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

A: The book "The Little Prince" was written by Antoine de Saint-Exupéry.


### 1-Shot

In [10]:
prompt = """Answer these questions:
Q: Who sang billie jean?
A: Michale Jackson
Q: Who wrote the book The Little Prince?
A: """

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Antoine de Saint-Exupéry


### 2. Chain-of-Thought
* 정답 사례 + 문제 해결 과정도 Prompt에 추가
* 답을 추리하는 과정을 적어준다.
* 대부분 프롬프트 엔지니어링 기법이 Chain-of-Thought


### Standard Prompting gpt-3.5, Not Chain-of-Thought

In [11]:
# Prompt 출처: https://github.com/microsoft/generative-ai-for-beginners/tree/main/05-advanced-prompts

prompt = """Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Alice would have 5 apples - 3 thrown + 2 given to Bob - 1 given back = 3 apples.


### Chain-of-Thought Prompting gpt-3.5

In [12]:
prompt = """Lisa has 7 apples, throws 1 apple, gives 4 apples to Bart and Bart gives one back:
7 - 1 = 6
6 - 4 = 2
2 + 1 = 3

Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Alice has 4 apples left. 

5 - 3 = 2
2 - 2 = 0
0 + 1 = 1


* Standard Prompting은 문제를 못풀었지만, Chain-of-Thought Prompting도 여러번 해보면 풀었지만, 못 푼 경우도 있다.
* Standard Prompting gpt-3.5는 문제를 못풀었지만, gpt-4는 문제를 풀 수 있는지 알아보자

### Standard Prompting gpt-4, Not Chain-of-Thought

In [13]:
# Prompt 출처: https://github.com/microsoft/generative-ai-for-beginners/tree/main/05-advanced-prompts

prompt = """Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"""

completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Let's break down the sequence of events to determine how many apples Alice has at the end:

1. Alice starts with 5 apples.
2. Alice throws 3 apples away. Now, she has \(5 - 3 = 2\) apples.
3. Alice gives 2 apples to Bob. Now, she has \(2 - 2 = 0\) apples.
4. Bob gives 1 apple back to Alice. Now, she has \(0 + 1 = 1\) apple.

Therefore, Alice has 1 apple.


### 결론
* Not Chain-of-Thought gpt-3.5은 풀지 못했고, Chani-of-Thought gpt-3.5는 푼 경우도 있고 아닌 경우도 있다.
* gpt-4는 문제를 풀 수 있다.
* gpt-4에는 Chain-of-Thought 상태가 녹아있다는 것을 알 수 있다.
* 예전에는 어떤 프롬프트를 쓰면 된다는 점이 있었지만, 요새는 모델이 좋아 좋은 모델을 쓰면 된다.
* 그러나 gpt-4는 gpt-3.5보다 30배 정도 비싸서..ㅎㅎ

### Summary
1. ChatGPT에게 자연스럽게 질문하는 방법 'Zero-Shot Prompting'
2. Zero-Shot보다 더 좋은 방법론은 'Few-Shot', 'Chain-of-Thought'
3. Few-Shot은 Prompt에 예시 추가
4. Chain-of-Thought은 여기에 정답 추리 과정까지 추가
5. Chain-of-Thought은 출력값 Voting 앙상블

## 프롬프트 고도화

### 1. KKMLU 한글 객관식 Question

In [14]:
# 정답은 B=6

question = 'x, y가 세 부등식 y ≤ x+3, y ≤ -4x+3, y ≥ 0을 만족할 때, x+y의 최댓값을 M, 최솟값을 m이라 하면 M-m의 값은?'

A = 4
B = 6
C = 8
D = 10

### GPT-3.5

In [15]:
# Prompt 출처: KMMLU 논문에서 실제로 평가에 사용한 Prompt (논문: https://arxiv.org/pdf/2402.11548.pdf)

prompt = f"""{question}
A. {A}
B. {B}
C. {C}
D. {D}
정답："""

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

C. 8

먼저 y ≤ x+3과 y ≤ -4x+3을 만족하는 영역을 그래프로 그리면 삼각형이 되고, y ≥ 0을 만족하는 부분은 x축 이하의 영역이 됩니다. 이 삼각형의 꼭지점은 (0,3), (0,0), (-3,0)이므로 이 삼각형의 꼭지점을 이용하여 x+y의 최댓값과 최솟값을 구할 수 있습니다.

(0,3)에서 x+y의 값은 3, (0,0)에서 x+y의 값은 0, (-3,0)에서 x+y의 값은 -3이므로 x+y의 최댓값은 3이고 최솟값은 -3입니다. 따라서 M-m = 3 - (-3) = 6이므로 정답은 C. 8입니다.


### GPT-4

In [16]:
# Prompt 출처: KMMLU 논문에서 실제로 평가에 사용한 Prompt (논문: https://arxiv.org/pdf/2402.11548.pdf)

prompt = f"""{question}
A. {A}
B. {B}
C. {C}
D. {D}
정답："""

completion = client.chat.completions.create(
    model='gpt-4-turbo-2024-04-09',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

이 문제를 해결하기 위해 주어진 부등식들을 이용하여 영역을 그래프에 나타내고, 이 영역의 꼭짓점에서 x+y의 최댓값과 최솟값을 찾아야 합니다.

1. **부등식 그리기**:
   - \( y \leq x + 3 \)
   - \( y \leq -4x + 3 \)
   - \( y \geq 0 \)

2. **교차점 찾기**:
   - \( y = x + 3 \)와 \( y = -4x + 3 \)의 교차점을 찾습니다.
     \[ x + 3 = -4x + 3 \]
     \[ 5x = 0 \]
     \[ x = 0 \]
     \[ y = 0 + 3 = 3 \]
     교차점은 (0, 3)입니다.

   - \( y = x + 3 \)와 \( y = 0 \)의 교차점을 찾습니다.
     \[ 0 = x + 3 \]
     \[ x = -3 \]
     교차점은 (-3, 0)입니다.

   - \( y = -4x + 3 \)와 \( y = 0 \)의 교차점을 찾습니다.
     \[ 0 = -4x + 3 \]
     \[ 4x = 3 \]
     \[ x = \frac{3}{4} \]
     교차점은 (\(\frac{3}{4}\), 0)입니다.

3. **x+y의 값 계산**:
   - 점 (-3, 0)에서 \( x + y = -3 + 0 = -3 \)
   - 점 (0, 3)에서 \( x + y = 0 + 3 = 3 \)
   - 점 (\(\frac{3}{4}\), 0)에서 \( x + y = \frac{3}{4} + 0 = \frac{3}{4} \)

4. **최댓값과 최솟값**:
   - 최댓값 \( M = 3 \)
   - 최솟값 \( m = -3 \)

5. **M - m 계산**:
   \[ M - m = 3 - (-3) = 3 + 3 = 6 \]

따라서, \( M - m \)의 값은 6입니다. 정답은 **B. 6**입니다.


### GPT-3.5 고도화 1

In [20]:
# Prompt 출처: KMMLU 논문에서 실제로 평가에 사용한 Prompt (논문: https://arxiv.org/pdf/2402.11548.pdf)

prompt = f"""You are an Professional in Mathematics. Below is given a math question in Korean.

{question}
A. {A}
B. {B}
C. {C}
D. {D}
Answer："""

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

C. 8


### gpt-3.5 고도화 2
* 차근차근 설명해주면서 생각할 시간을 준다는 마음으로 작성

In [22]:
# Prompt 출처: KMMLU 논문에서 실제로 평가에 사용한 Prompt (논문: https://arxiv.org/pdf/2402.11548.pdf)

prompt = f"""You are a Professional in Mathematics. Below is given a math question in Korean.
You are to think carefully about the question and choose one of four given answers. Only one of them is true.

{question}
A. {A}
B. {B}
C. {C}
D. {D}
Answer:"""

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

C. 8


In [23]:
# Prompt 출처: KMMLU 논문에서 실제로 평가에 사용한 Prompt (논문: https://arxiv.org/pdf/2402.11548.pdf)

prompt = f"""You are a Professional in Mathematics. Below is given a math question in Korean.
You are to think carefully about the question and choose one of four given answers. Only one of them is true.
Give reasons about why you think your answer is correct.

{question}
A. {A}
B. {B}
C. {C}
D. {D}
Answer："""

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

B. 6

To find the maximum and minimum values of x+y, we need to find the intersection points of the given inequalities. 

First, we find the intersection points of y ≤ x+3 and y ≤ -4x+3:
x+3 = -4x+3
5x = 0
x = 0

When x = 0, y = 3

Next, we find the intersection points of y ≤ x+3 and y ≥ 0:
x+3 = 0
x = -3

When x = -3, y = 0

Therefore, the maximum value of x+y is when x = 0 and y = 3, which is 3. The minimum value of x+y is when x = -3 and y = 0, which is -3. 

Therefore, M-m = 3 - (-3) = 6. 

Thus, the correct answer is B. 6.


### 2. Open Ended 태스크
평가기준
1. G-Eval (Model-Based Evaluation)
2. Rouge (Code-Based Evaluation)

### 테스트 케이스

In [24]:
# https://v.daum.net/v/20240417080004859

news = """한국프로야구(KBO) 중계권은 티빙이 가져갔지만, 네이버(035420)와 카카오(035720)는 중계를 보지 않아도 야구를 즐길 수 있는 서비스를 강화하고 있다. 네이버는 '오픈톡', 카카오는 '프로야구봇'으로 이용자를 끌어들이는 데 성공했다.

17일 네이버에 따르면 프로야구 10개 구단 '오픈톡' 참여자 수는 최근 일주일(중복 제외) 동안 100만 명에 달한다.

오픈톡은 실시간으로 정보를 공유하고 의견을 주고받는 커뮤니티 서비스다. 한화구단의 오픈톡에만 29만 명이 넘게 참여하고 있다.

또 네이버는 올해 2월부터 스포츠 뉴스에 알고리즘 기능을 도입했다. 야구를 좋아하는 이용자들이 더 많은 야구 뉴스를 볼 수 있도록 배려했다.

최근 클릭한 기사, 선호 언론사 등을 분석해 이용자가 좋아하는 뉴스를 우선적으로 노출하는 방식이다.

카카오는 카카오톡 채널 '프로야구봇' 서비스를 강화하고 있다. 16일 기준 프로야구봇을 친구추가한 이용자는 약 35만 명이다.

프로야구봇을 친구로 추가하면 응원팀의 소식, 라인업, 경기기록 등 프로야구와 관련한 소식을 공유받을 수 있다.

특히 실시간 경기 상황을 바로바로 보내주는 '톡중계'를 이용해 실시간 중계를 보지 못해도 경기의 흐름을 알 수 있다.

올해 시즌부터는 '경기결과 요약', '1군 등록·말소 정보'도 추가로 제공한다. 경기결과 요약은 경기를 못 본 날에도 주요 흐름 파악에 문제없을 정도로 자세하다.

'우리팀 환호 알림' 베타 서비스도 시작했다. 경기 중 팬들이 환호하는 순간을 푸시 메시지로 알려주는 것이다.

다음(Daum)스포츠에서는 '그래픽 중계'를 시작했다. 투구 정보와 출루 상황 등 경기 상황을 그래픽으로 제공해 영상 중계 없이도 현장감 있는 경기를 즐길 수 있도록 했다.

카카오는 "더 즐거운 야구 생활을 할 수 있도록 여러 가지 기능을 많이 준비하고 있다"고 말했다."""

### GPT-3.5

In [25]:
prompt = f"""아래 뉴스 기사를 3줄로 요약해줘.

{news}
"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

summary_gpt3 = completion.choices[0].message.content
print(summary_gpt3)

네이버와 카카오, KBO 중계권을 놓고 경쟁 중이지만 야구 팬들을 위한 서비스를 강화 중. 네이버의 '오픈톡'과 카카오의 '프로야구봇'으로 이용자들을 끌어들이는 데 성공. 또한 네이버는 알고리즘 기능을 도입해 야구 뉴스를 더 많이 제공하고, 카카오는 '톡중계'와 '그래픽 중계'를 통해 현장감 있는 야구 경기를 즐길 수 있도록 노력 중.


In [26]:
for line in summary_gpt3.split('. '):
  print(line)

네이버와 카카오, KBO 중계권을 놓고 경쟁 중이지만 야구 팬들을 위한 서비스를 강화 중
네이버의 '오픈톡'과 카카오의 '프로야구봇'으로 이용자들을 끌어들이는 데 성공
또한 네이버는 알고리즘 기능을 도입해 야구 뉴스를 더 많이 제공하고, 카카오는 '톡중계'와 '그래픽 중계'를 통해 현장감 있는 야구 경기를 즐길 수 있도록 노력 중.


### GPT-4

In [27]:
prompt = f"""아래 뉴스 기사를 3줄로 요약해줘.

{news}
"""

completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

summary_gpt4 = completion.choices[0].message.content
print(summary_gpt4)

1. 티빙이 KBO 중계권을 확보했으나, 네이버와 카카오는 각각 '오픈톡'과 '프로야구봇' 서비스를 통해 사용자들이 중계를 보지 않고도 야구 경기를 즐길 수 있도록 서비스를 강화하고 있다.
2. 네이버의 '오픈톡'은 실시간 정보 공유와 의견 교환을 가능하게 하는 커뮤니티 서비스로, 최근 일주일간 프로야구 10개 구단 참여자 수가 100만 명에 달하며, 한화 구단의 오픈톡 참여자만 29만 명이 넘는다.
3. 카카오의 '프로야구봇'은 카카오톡 채널을 통해 응원팀의 소식, 라인업, 경기기록 등을 제공하며, 실시간 '톡중계' 기능을 통해 경기의 흐름을 파악할 수 있으며, 추가적으로 경기결과 요약과 1군 등록·말소 정보도 제공한다.


### 뉴스기사 평가 프롬프트 GPT-3.5 [G-Eval]

In [28]:
eval_prompt = f"""You will be given one summary written for a news article.
Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions carefully. Please keep this
document open while reviewing, and refer to it as needed.
Evaluation Criteria:
Coherence (1-5) - the collective quality of all sentences. We align this dimension with
the DUC quality question of structure and coherence whereby ”the summary should be
well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic.”
Evaluation Steps:
1. Read the news article carefully and identify the main topic and key points.
2. Read the summary and compare it to the news article. Check if the summary covers the main
topic and key points of the news article, and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest
based on the Evaluation Criteria.
Example:
Source Text:
{news}
Summary:
{summary_gpt3}
Evaluation Form (scores ONLY):
- Coherence:"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": eval_prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

4


### 뉴스기사 평가 프롬프트 gpt-4 [G-Eval]

In [29]:
eval_prompt = f"""You will be given one summary written for a news article.
Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions carefully. Please keep this
document open while reviewing, and refer to it as needed.
Evaluation Criteria:
Coherence (1-5) - the collective quality of all sentences. We align this dimension with
the DUC quality question of structure and coherence whereby ”the summary should be
well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic.”
Evaluation Steps:
1. Read the news article carefully and identify the main topic and key points.
2. Read the summary and compare it to the news article. Check if the summary covers the main
topic and key points of the news article, and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest
based on the Evaluation Criteria.
Example:
Source Text:
{news}
Summary:
{summary_gpt4}
Evaluation Form (scores ONLY):
- Coherence:"""

completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[{"role": "user", "content": eval_prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

- Coherence: 5


### 평가 Rouge

In [30]:
from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(summary_gpt3, summary_gpt4)
scores

[{'rouge-1': {'r': 0.19736842105263158,
   'p': 0.35714285714285715,
   'f': 0.2542372835507039},
  'rouge-2': {'r': 0.0625, 'p': 0.11363636363636363, 'f': 0.08064515671175884},
  'rouge-l': {'r': 0.18421052631578946,
   'p': 0.3333333333333333,
   'f': 0.23728813100833102}}]

### 고도화 GPT-3.5

In [31]:
prompt = f"""You are a helpful Summarization Assistant. Below is given a news in Korean.
Please summarize below news in three sentences. You summary should follow below rules:

1. You summary should not contain any information that is not present in the news.

{news}
"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

summary_gpt3_v2 = completion.choices[0].message.content
print(summary_gpt3_v2)

네이버와 카카오는 KBO 중계권을 잃었지만, '오픈톡'과 '프로야구봇'을 통해 야구 팬들에게 서비스를 제공하고 있다. 네이버의 '오픈톡'은 10개 구단의 참여자 수가 최근 일주일 동안 100만 명에 이르며, 한화구단의 오픈톡에는 29만 명이 참여하고 있다. 카카오는 '프로야구봇'을 통해 약 35만 명의 이용자에게 응원팀의 소식과 실시간 경기 상황을 제공하고 있다.


* 개선된 모습을 보임 (경쟁중 -> 잃었지만), 원래 잃었다

### Few-Shot Prompting (1-Shot)

In [32]:
sample_news = """한국야구위원회(KBO) 리그 정규 시즌이 23일 개막하는 가운데, 카카오가 포털 다음에서 시즌 모든 경기에 대해 처음으로 그래픽 중계를 진행한다.

그래픽 중계는 실시간 경기 상황을 그래픽으로 보여주는 LMT(Live Match Tracker) 서비스다.

투구 정보와 출루 상황 등의 경기 상황을 그래픽으로 제공해 영상 중계 없이도 현장감 있는 경기를 즐길 수 있는 것이 특징이다.

다음 스포츠 내 KBO 게임센터에서는 문자 중계, 경기 일정·기록, 뉴스 등의 콘텐츠도 함께 확인할 수 있다.

카카오 다음CIC는 지난 1월 해외 축구 게임센터에 그래픽 중계를 처음으로 도입했다.

이후 2023 아시안컵과 국내 프로축구(K리그)에 그래픽 중계를 적용했다.

카카오는 향후 다음에서 미국 프로야구(MLB), 미국 프로농구(NBA), 한국 프로농구(KBL) 등 다양한 스포츠 경기에 그래픽 중계를 확대 적용할 계획이다.

이 밖에 카카오톡에서는 '프로야구봇' 채널도 추가할 수 있다.

채널을 추가하면 응원팀의 소식, 라인업, 경기 기록 등 프로야구와 관련한 소식들을 톡 채널로 공유받을 수 있다.

또 현재 경기 상황을 한눈에 파악할 수 있는 카드형 서비스 '두근두근'과 승부 예측 기능도 있다.

2024시즌부터는 '경기 결과 요약', '1군 등록·말소 정보'도 추가로 제공한다."""

prompt = f"""아래 뉴스 기사를 3줄로 요약해줘.

{sample_news}
"""

completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

sample_summary = completion.choices[0].message.content
print(sample_summary)

카카오가 포털 다음을 통해 KBO 리그의 모든 경기에 대해 처음으로 그래픽 중계를 제공하며, 이는 실시간 경기 상황을 그래픽으로 보여주는 LMT(Live Match Tracker) 서비스를 포함한다. 다음 스포츠의 KBO 게임센터에서는 문자 중계, 경기 일정, 기록, 뉴스 등 다양한 콘텐츠를 제공하고, 카카오는 향후 MLB, NBA, KBL 등 다른 스포츠 경기에도 그래픽 중계를 확대 적용할 계획이다. 추가적으로 카카오톡에서는 '프로야구봇' 채널을 통해 응원팀 소식과 경기 기록 등을 받아볼 수 있으며, 2024시즌부터는 경기 결과 요약과 선수 등록 정보도 제공할 예정이다.


In [33]:
for line in sample_summary.split('. '):
  print(line)

카카오가 포털 다음을 통해 KBO 리그의 모든 경기에 대해 처음으로 그래픽 중계를 제공하며, 이는 실시간 경기 상황을 그래픽으로 보여주는 LMT(Live Match Tracker) 서비스를 포함한다
다음 스포츠의 KBO 게임센터에서는 문자 중계, 경기 일정, 기록, 뉴스 등 다양한 콘텐츠를 제공하고, 카카오는 향후 MLB, NBA, KBL 등 다른 스포츠 경기에도 그래픽 중계를 확대 적용할 계획이다
추가적으로 카카오톡에서는 '프로야구봇' 채널을 통해 응원팀 소식과 경기 기록 등을 받아볼 수 있으며, 2024시즌부터는 경기 결과 요약과 선수 등록 정보도 제공할 예정이다.


In [34]:
prompt = f"""You are a helpful Summarization Assistant. Below is give a news in Korean.
Please summarize below news in three sentences. Your summary should not contain any information that is not present in the news.

[Sample News Start]
{sample_news}
[Sample News End]

[Sample Summary Start]
{sample_summary}
[Sample Summary End]

News:
{news}

Summary:"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

summary_gpt3_v2 = completion.choices[0].message.content
print(summary_gpt3_v2)

네이버와 카카오는 KBO 중계권을 놓치고 있지만, '오픈톡'과 '프로야구봇'을 통해 야구 팬들에게 서비스를 제공하고 있다. 네이버의 '오픈톡'은 100만 명 이상의 참여자가 있는 커뮤니티 서비스이며, 카카오의 '프로야구봇'은 약 35만 명의 이용자가 있다. 또한 다음스포츠에서는 '그래픽 중계'를 시작하여 영상 중계 없이도 현장감 있는 경기를 즐길 수 있도록 했다.


In [35]:
for line in summary_gpt3_v2.split('. '):
  print(line)

네이버와 카카오는 KBO 중계권을 놓치고 있지만, '오픈톡'과 '프로야구봇'을 통해 야구 팬들에게 서비스를 제공하고 있다
네이버의 '오픈톡'은 100만 명 이상의 참여자가 있는 커뮤니티 서비스이며, 카카오의 '프로야구봇'은 약 35만 명의 이용자가 있다
또한 다음스포츠에서는 '그래픽 중계'를 시작하여 영상 중계 없이도 현장감 있는 경기를 즐길 수 있도록 했다.


### Rouge 평가

In [36]:
from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(summary_gpt3_v2, summary_gpt4)
scores

[{'rouge-1': {'r': 0.25, 'p': 0.4418604651162791, 'f': 0.31932772647694374},
  'rouge-2': {'r': 0.0875, 'p': 0.1590909090909091, 'f': 0.1129032212278878},
  'rouge-l': {'r': 0.21052631578947367,
   'p': 0.37209302325581395,
   'f': 0.26890755840971686}}]

* 코드 기반 평가방법으로는 많이 나아졌다.

### G-Eval

In [37]:
eval_prompt = f"""You will be given one summary written for a news article.
Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions carefully. Please keep this
document open while reviewing, and refer to it as needed.
Evaluation Criteria:
Coherence (1-5) - the collective quality of all sentences. We align this dimension with
the DUC quality question of structure and coherence whereby ”the summary should be
well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic.”
Evaluation Steps:
1. Read the news article carefully and identify the main topic and key points.
2. Read the summary and compare it to the news article. Check if the summary covers the main
topic and key points of the news article, and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest
based on the Evaluation Criteria.
Example:
Source Text:
{news}
Summary:
{summary_gpt3_v2}
Evaluation Form (scores ONLY):
- Coherence:"""

completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[{"role": "user", "content": eval_prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

- Coherence: 4
